# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../da…",0.983033,"""84488899730106…","""15403709084833…","""16213955297970…","""75129883939917…",false
"""../../../../da…",0.144221,"""16738404728860…","""15035238798132…","""16213955297970…","""14170067642315…",true
"""../../../../da…",0.679745,"""17235298700813…","""28235007362858…","""35286693972771…","""75129883939917…",false
"""../../../../da…",1.0,"""16781042313839…","""15035238798132…","""13401532865605…","""75129883939917…",false
"""../../../../da…",0.718982,"""16934794238391…","""15403709084833…","""13401532865605…","""96357049266983…",true
"""../../../../da…",0.0,"""49326568206307…","""28235007362858…","""92695101123115…",null,false
"""../../../../da…",0.02333,"""55874148604335…","""28235007362858…","""13401532865605…","""96357049266983…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""86637905523248…","""14504443648784…","""FirstMortgage3…","""12963358137711…","""Rejected"""
"""../../../../da…",0.144221,"""18085710732903…","""11428482888644…","""FirstMortgage3…","""48838957026443…","""Accepted"""
"""../../../../da…",0.679745,"""16741786131439…","""82964748011493…","""MoneyMarketSav…","""12963358137711…","""Rejected"""
"""../../../../da…",1.0,"""41817579706788…","""11428482888644…","""BasicChecking""","""12963358137711…","""Rejected"""
"""../../../../da…",0.718982,"""38627314990843…","""14504443648784…","""BasicChecking""","""27529608056715…","""Accepted"""
"""../../../../da…",0.0,"""58860461245134…","""82964748011493…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""67919365267249…","""82964748011493…","""BasicChecking""","""27529608056715…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""12651133628755…","""45176458495062…","""FirstMortgage3…","""12659370993736…","""Rejected"""
"""../../../../da…",0.144221,"""76878682585658…","""51228893375897…","""FirstMortgage3…","""51213403744729…","""Accepted"""
"""../../../../da…",0.679745,"""17069499925737…","""37851203220612…","""MoneyMarketSav…","""12659370993736…","""Rejected"""
"""../../../../da…",1.0,"""14443304822234…","""51228893375897…","""BasicChecking""","""12659370993736…","""Rejected"""
"""../../../../da…",0.718982,"""14600805697688…","""45176458495062…","""BasicChecking""","""19325612373511…","""Accepted"""
"""../../../../da…",0.0,"""17473884125598…","""37851203220612…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""16538267585347…","""37851203220612…","""BasicChecking""","""19325612373511…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""14384407981425…","""85772107202178…","""FirstMortgage3…","""84937232671467…","""Rejected"""
0.144221,"""44567362412308…","""17308166830456…","""FirstMortgage3…","""76458015161407…","""Accepted"""
0.679745,"""82079698818724…","""17248607605095…","""MoneyMarketSav…","""84937232671467…","""Rejected"""
1.0,"""45075160691482…","""17308166830456…","""BasicChecking""","""84937232671467…","""Rejected"""
0.718982,"""12013953044440…","""85772107202178…","""BasicChecking""","""77734984076201…","""Accepted"""
0.0,"""82701408024061…","""17248607605095…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""15948992459756…","""17248607605095…","""BasicChecking""","""77734984076201…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""c73023dda51de1…","""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.144221,"""5414b197fb87b3…","""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""../../../../da…",0.679745,"""892f5d61a9215e…","""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",1.0,"""c3aa807d9e927a…","""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.718982,"""c2d492f9a05073…","""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""../../../../da…",0.0,"""b21059221605c4…","""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""80cbe489dcecd4…","""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
